<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#notes-and-questions" data-toc-modified-id="notes-and-questions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>notes and questions</a></span></li><li><span><a href="#Setup" data-toc-modified-id="Setup-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setup</a></span><ul class="toc-item"><li><span><a href="#Setup---Debug" data-toc-modified-id="Setup---Debug-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup - Debug</a></span></li><li><span><a href="#Setup---Imports" data-toc-modified-id="Setup---Imports-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Setup - Imports</a></span></li><li><span><a href="#Setup---working-folder-paths" data-toc-modified-id="Setup---working-folder-paths-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Setup - working folder paths</a></span></li><li><span><a href="#Setup---logging" data-toc-modified-id="Setup---logging-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Setup - logging</a></span></li><li><span><a href="#Setup---virtualenv-jupyter-kernel" data-toc-modified-id="Setup---virtualenv-jupyter-kernel-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Setup - virtualenv jupyter kernel</a></span></li><li><span><a href="#Setup---Initialize-Django" data-toc-modified-id="Setup---Initialize-Django-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Setup - Initialize Django</a></span></li><li><span><a href="#Setup---ExportToContext-instance" data-toc-modified-id="Setup---ExportToContext-instance-2.7"><span class="toc-item-num">2.7&nbsp;&nbsp;</span>Setup - ExportToContext instance</a></span></li><li><span><a href="#Setup---Initialize-LoggingHelper" data-toc-modified-id="Setup---Initialize-LoggingHelper-2.8"><span class="toc-item-num">2.8&nbsp;&nbsp;</span>Setup - Initialize LoggingHelper</a></span></li></ul></li><li><span><a href="#load-sourcenet-data-into-context" data-toc-modified-id="load-sourcenet-data-into-context-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>load sourcenet data into context</a></span><ul class="toc-item"><li><span><a href="#Retrieve-Article-instances" data-toc-modified-id="Retrieve-Article-instances-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Retrieve Article instances</a></span></li><li><span><a href="#build-load-code-and-unit-tests" data-toc-modified-id="build-load-code-and-unit-tests-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>build load code and unit tests</a></span><ul class="toc-item"><li><span><a href="#Testing" data-toc-modified-id="Testing-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Testing</a></span></li></ul></li><li><span><a href="#Load-data-from-Article-instances" data-toc-modified-id="Load-data-from-Article-instances-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load data from Article instances</a></span></li></ul></li></ul></div>

# notes and questions

- Back to [Table of Contents](#Table-of-Contents)

Notes:

- probably will need a class that is an Article_Data container - reference to Article_Data, and then all the logic to process the entities and relations contained within.

Questions:

- Do we want an Identifier Type separate from Entity and Relation identifiers?  I think we do, so we can specify the entity type(s) a given identifier should be used on.

    - Created abstract one, and created a concrete entity identifier type.  Will create one for relation identifiers if needed.

# Setup

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
me = "sourcenet-to-context"

## Setup - Debug

- Back to [Table of Contents](#Table-of-Contents)

In [2]:
debug_flag = False

## Setup - Imports

- Back to [Table of Contents](#Table-of-Contents)

In [3]:
import datetime
from django.db.models import Avg, Max, Min
from django.utils.text import slugify
import json
import logging
import six

## Setup - working folder paths

- Back to [Table of Contents](#Table-of-Contents)

In [4]:
%pwd

'/home/jonathanmorgan/work/django/research/work/phd_work/analysis'

In [5]:
# current working folder
current_working_folder = "/home/jonathanmorgan/work/django/research/work/phd_work/analysis"
current_datetime = datetime.datetime.now()
current_date_string = current_datetime.strftime( "%Y-%m-%d-%H-%M-%S" )

## Setup - logging

- Back to [Table of Contents](#Table-of-Contents)

configure logging for this notebook's kernel (If you do not run this cell, you'll get the django application's logging configuration.

In [6]:
logging_file_name = "{}/{}-{}.log.txt".format( current_working_folder, me, current_date_string )
logging.basicConfig(
    level = logging.INFO,
    format = '%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    filename = logging_file_name,
    filemode = 'w' # set to 'a' if you want to append, rather than overwrite each time.
)
print( "Logging initialized, to {}".format( logging_file_name ) )

Logging initialized, to /home/jonathanmorgan/work/django/research/work/phd_work/analysis/sourcenet-to-context-2019-11-26-16-41-45.log.txt


## Setup - virtualenv jupyter kernel

- Back to [Table of Contents](#Table-of-Contents)

If you are using a virtualenv, make sure that you:

- have installed your virtualenv as a kernel.
- choose the kernel for your virtualenv as the kernel for your notebook (Kernel --> Change kernel).

Since I use a virtualenv, need to get that activated somehow inside this notebook.  One option is to run `../dev/wsgi.py` in this notebook, to configure the python environment manually as if you had activated the `sourcenet` virtualenv.  To do this, you'd make a code cell that contains:

    %run ../dev/wsgi.py
    
This is sketchy, however, because of the changes it makes to your Python environment within the context of whatever your current kernel is.  I'd worry about collisions with the actual Python 3 kernel.  Better, one can install their virtualenv as a separate kernel.  Steps:

- activate your virtualenv:

        workon research

- in your virtualenv, install the package `ipykernel`.

        pip install ipykernel

- use the ipykernel python program to install the current environment as a kernel:

        python -m ipykernel install --user --name <env_name> --display-name "<display_name>"
        
    `sourcenet` example:
    
        python -m ipykernel install --user --name sourcenet --display-name "research (Python 3)"
        
More details: [http://ipython.readthedocs.io/en/stable/install/kernel_install.html](http://ipython.readthedocs.io/en/stable/install/kernel_install.html)

## Setup - Initialize Django

- Back to [Table of Contents](#Table-of-Contents)

First, initialize my dev django project, so I can run code in this notebook that references my django models and can talk to the database using my project's settings.

In [7]:
# init django
django_init_folder = "/home/jonathanmorgan/work/django/research/work/phd_work"
django_init_path = "django_init.py"
if( ( django_init_folder is not None ) and ( django_init_folder != "" ) ):
    
    # add folder to front of path.
    django_init_path = "{}/{}".format( django_init_folder, django_init_path )
    
#-- END check to see if django_init folder. --#

In [8]:
%run $django_init_path

django initialized at 2019-11-26 16:41:50.635059


In [9]:
# context imports
from context.models import Entity
from context.models import Entity_Identifier_Type
from context.models import Entity_Identifier
from context.models import Entity_Type

# context_text imports
from context_text.article_coding.article_coding import ArticleCoder
from context_text.article_coding.article_coding import ArticleCoding
from context_text.article_coding.open_calais_v2.open_calais_v2_article_coder import OpenCalaisV2ArticleCoder
from context_text.collectors.newsbank.newspapers.GRPB import GRPB
from context_text.collectors.newsbank.newspapers.DTNB import DTNB
from context_text.export.to_context_base.export_to_context import ExportToContext
from context_text.models import Article
from context_text.models import Article_Subject
from context_text.models import Newspaper
from context_text.shared.context_text_base import ContextTextBase

## Setup - ExportToContext instance

- Back to [Table of Contents](#Table-of-Contents)

Make instance, set instance variables.

In [10]:
my_exporter = ExportToContext()

# no variables to set, yet...
my_exporter.set_article_uuid_id_type_name( ExportToContext.ENTITY_ID_TYPE_ARTICLE_NEWSBANK_ID )

<Entity_Identifier_Type: 4 - article_newsbank_id - Newsbank - ( article )>

## Setup - Initialize LoggingHelper

- Back to [Table of Contents](#Table-of-Contents)

Create a LoggingHelper instance to use to log debug and also print at the same time.

Preconditions: Must be run after Django is initialized, since `python_utilities` is in the django path.

In [11]:
# python_utilities
from python_utilities.logging.logging_helper import LoggingHelper

# init
my_logging_helper = LoggingHelper()
my_logging_helper.set_logger_name( me )
log_message = None

# load sourcenet data into context

- Back to [Table of Contents](#Table-of-Contents)

logic:

- get all Article_Data created by automated coder, open calais coder type.
- for each:

    - Article processing:
    
        - look for article entity in context.  If not created, create one of type "article".  Store the sourcenet ID as identifier of type "article_sourcenet_id".
        - Store the ID and entity reference for the Article (in object?  Should I make a place for the article to hold an Entity ID?).
        - check to see if the newspaper has an entity (using its sourcenet ID as the lookup identifier).  If not, create one.

    - Person processing:
    
        - Add all people who are either reporters/authors or subjects to `context` as entities of type "person", with identifier of type `person_sourcenet_id` set to their internal django/database "id", and with identifier of type `person_open_calais_uuid` set to their OpenCalais ID, if they have one.  If they have any other types of IDs, add them too, untyped.  Once entity is created, store ForeignKey of entity in Person record.
        - Author/Reporter processing:

            - look for each author's person entity.  If not found, create one.
    
        - Subject/Source processing:
    
            - look for a person entity for each subject/source.  If not found, create one.

    - Relations - create the following:
    
        - from newspaper
        
            - newspaper_reporter - Reporter at a newspaper, evidence of which is byline on an article in that newspaper. FROM newspaper TO person (reporter) THROUGH article.
            - newspaper_source - Person quoted in an article published by a newspaper. FROM newspaper TO person (source) THROUGH article.
            - newspaper_subject - Subject of an article published in a given newspaper. FROM newspaper TO person (subject, including sources) THROUGH article.
            - newspaper_article - Article published in a particular newspaper. FROM newspaper TO article.

        - from article

            - author - Author/Reporter of an article - FROM article TO reporter.
            - subject - Subject of a story. FROM article TO subject person.
            - source - Source quoted in an article - FROM article TO source person.

        - through article

            - article_container - Parent for relations based on entities being mentioned in the same article. To start, just people, but eventually, for example, could also include location.
            - mentioned - Mentioned in an article. FROM reporter/author TO subject THROUGH article.
            - quoted - The "from" person quoted the "to" person in a publication. FROM reporter TO source THROUGH article.
            - same_article_sources - Sources in the same article, FROM source person TO source person THROUGH article.
            - same_article_subjects - Two people who are in a particular article together (includes subjects and sources).
            - shared_byline - Shared Byline on an article - joint authors - FROM author TO author THROUGH article.

Relations broken out by person type:

- For all reporters/authors:

    - create a relation of type "`author`" between the article's entity (FROM) and the entity of the person (TO) for each author.
    - if multiple authors, for each pair of authors, create a relation of type "`shared_byline`" between the two (it is undirected), THROUGH the article.

- For all subjects, including sources:

    - create a relation of type "`subject`" between the article's entity (FROM) and the entity of the subject person (TO).
    - if multiple subjects, for each pair of subjects, create a relation of type "`same_article_subjects`" between the two (it is undirected), THROUGH the article.
    - create a relation of type "`mentioned`" between each of the article's authors (FROM) and the subject (TO), THROUGH the article.

- For all sources

    - create a relation of type "`source`" between the article's entity and the entity for the source person.
    - if multiple sources, for each pair of sources, create a relation of type "`same_article_sources`" between the two (it is undirected), THROUGH the article.
    - create a relation of type "`quoted`" between each of the article's authors (FROM) and the source (TO), THROUGH the article.


Convenience methods:

- method to find entity - based on type and identifier (accept all the fields that make sense, including optional identifier type instance).
- method to find relation - based on type, etc.

Notes:

- all ties are undirected.
- relations can have three foreign keys into Entity - FROM, TO, and THROUGH (for a containing relationship, like the article that included two sources that we are relating).

## Retrieve Article instances

- Back to [Table of Contents](#Table-of-Contents)

Before we do anything else, need to be able to pull back all the articles whose data we want to load into the context store.

In [12]:
# look for publications that have article data:
# - coded by automated coder
# - with coder type of "OpenCalais_REST_API_v2"

# get automated coder
automated_coder_user = ContextTextBase.get_automated_coding_user()

log_message = "{} - Loaded automated user: {}, id = {}".format( datetime.datetime.now(), automated_coder_user, automated_coder_user.id )
my_logging_helper.output_message( log_message, do_print_IN = True, log_level_code_IN = logging.INFO )

2019-11-26 16:42:00.546618 - Loaded automated user: automated, id = 2


In [13]:
# find articles with Article_Data created by the automated user...
article_qs = Article.objects.filter( article_data__coder = automated_coder_user )

# ...and specifically coded using OpenCalais V2...
article_qs = article_qs.filter( article_data__coder_type = OpenCalaisV2ArticleCoder.CONFIG_APPLICATION )

# ...and finally, we just want the distinct articles by ID.
article_qs = article_qs.order_by( "id" ).distinct( "id" )

# count?
article_count = article_qs.count()
log_message = "Found {} articles".format( article_count )
my_logging_helper.output_message( log_message, do_print_IN = True, log_level_code_IN = logging.INFO )

Found 43843 articles


## build load code and unit tests

- Back to [Table of Contents](#Table-of-Contents)

### Testing

- Back to [Table of Contents](#Table-of-Contents)

First, make unit tests to test convenience methods added to the following models, in `context/tests/models/`:

- // `Entity_Identifier_Type` - test_Entity_Identifier_Type_model.py
- // `Entity_Identifier` - test_Entity_Identifier_model.py
- // `Entity` - test_Entity_model.py

Also, // move instance creation class methods along with their constants over into "TestHelper" from test_Entity_Identifier_model.py, so they can be re-used across test classes.

To run: `python manage.py test context.tests`

In test data:

- article 21925:

    - Article_Data: `SELECT * FROM context_text_article_data WHERE article_id = 21925;`

TODO:

- test_entity_model.py

    - test_get_entity_for_identifier

- test_export_to_context.py

    - test_get_article_uuid_id_type
    - test_set_article_uuid_id_type_name

## Load data from Article instances

- Back to [Table of Contents](#Table-of-Contents)

Now, we actually load the data...

In [ ]:
result = my_exporter.process_articles( article_qs )

In create_q_filter_automated_by_coder_type: automated coder user: 2 - automated
----> Processed 100 of 43843 articles at 2019-11-26 16:44:10.194920 ( elapsed: 0:02:04.258806; average: 0:00:01.242588 )
----> Processed 200 of 43843 articles at 2019-11-26 16:46:08.333038 ( elapsed: 0:04:02.396924; average: 0:00:01.211985 )
----> Processed 300 of 43843 articles at 2019-11-26 16:48:03.126190 ( elapsed: 0:05:57.190076; average: 0:00:01.190634 )
----> Processed 400 of 43843 articles at 2019-11-26 16:50:09.831284 ( elapsed: 0:08:03.895170; average: 0:00:01.209738 )
----> Processed 500 of 43843 articles at 2019-11-26 16:52:15.175648 ( elapsed: 0:10:09.239534; average: 0:00:01.218479 )
----> Processed 600 of 43843 articles at 2019-11-26 16:54:02.125858 ( elapsed: 0:11:56.189744; average: 0:00:01.193650 )
----> Processed 700 of 43843 articles at 2019-11-26 16:55:50.575779 ( elapsed: 0:13:44.639665; average: 0:00:01.178057 )
----> Processed 800 of 43843 articles at 2019-11-26 16:57:46.909539 ( ela

----> Processed 6800 of 43843 articles at 2019-11-26 20:15:53.821748 ( elapsed: 3:33:47.885634; average: 0:00:01.886454 )
----> Processed 6900 of 43843 articles at 2019-11-26 20:19:51.879550 ( elapsed: 3:37:45.943436; average: 0:00:01.893615 )
----> Processed 7000 of 43843 articles at 2019-11-26 20:23:44.463678 ( elapsed: 3:41:38.527564; average: 0:00:01.899790 )
----> Processed 7100 of 43843 articles at 2019-11-26 20:28:01.227684 ( elapsed: 3:45:55.291570; average: 0:00:01.909196 )
----> Processed 7200 of 43843 articles at 2019-11-26 20:34:57.971819 ( elapsed: 3:52:52.035705; average: 0:00:01.940561 )
----> Processed 7300 of 43843 articles at 2019-11-26 20:39:36.914506 ( elapsed: 3:57:30.978392; average: 0:00:01.952189 )
----> Processed 7400 of 43843 articles at 2019-11-26 20:44:19.426220 ( elapsed: 4:02:13.490106; average: 0:00:01.963985 )
----> Processed 7500 of 43843 articles at 2019-11-26 20:48:00.090877 ( elapsed: 4:05:54.154763; average: 0:00:01.967221 )
----> Processed 7600 of 

----> Processed 13500 of 43843 articles at 2019-11-27 00:41:51.046239 ( elapsed: 7:59:45.110125; average: 0:00:02.132230 )
----> Processed 13600 of 43843 articles at 2019-11-27 00:45:35.271025 ( elapsed: 8:03:29.334911; average: 0:00:02.133039 )
----> Processed 13700 of 43843 articles at 2019-11-27 00:49:50.928313 ( elapsed: 8:07:44.992199; average: 0:00:02.136131 )
----> Processed 13800 of 43843 articles at 2019-11-27 00:54:11.054125 ( elapsed: 8:12:05.118011; average: 0:00:02.139501 )
----> Processed 13900 of 43843 articles at 2019-11-27 00:58:14.435663 ( elapsed: 8:16:08.499549; average: 0:00:02.141619 )
----> Processed 14000 of 43843 articles at 2019-11-27 01:01:54.309626 ( elapsed: 8:19:48.373512; average: 0:00:02.142027 )
----> Processed 14100 of 43843 articles at 2019-11-27 01:04:58.391259 ( elapsed: 8:22:52.455145; average: 0:00:02.139890 )
----> Processed 14200 of 43843 articles at 2019-11-27 01:09:26.276984 ( elapsed: 8:27:20.340870; average: 0:00:02.143686 )
----> Processed 